In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=2f8c12241d4d1aa7db318ff41357330132456d3ea998956f4461656a30d13280
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pandas as pd

spark, который создаст эту таблицу (данные указаны ниже) и в результате выдаст таблицу в которой будут имена сотрудников, которые зарабатывают больше своих менеджеров.
Данные для таблицы:
data = {'id': [1, 2, 3, 4], Name': ['Joe', 'Henry', Sam', 'Max'],
Salary':['70', 80', 60', 90], Managerld :['3', 4 , Null', Null]}


Создаем сессию

In [4]:
spark = SparkSession.builder.appName('name_not_manager_with_big_sallary').getOrCreate()


Объявление серии

In [7]:
data = {'id': [1, 2, 3, 4], 'Name': ['Joe', 'Henry', 'Sam', 'Max'],
         'Salary':['70', '80', '60', '90'], 'Managerld':['3', '4' , 'Null', 'Null']}
df = pd.DataFrame(data)
df

,id,Name,Salary,Managerld
0,1,Joe,70,3
1,2,Henry,80,4
2,3,Sam,60,Null
3,4,Max,90,Null


Собираем id мэнэджеров

In [32]:
df_managers = df['Managerld'].to_list()
df_managers

['3', '4', 'Null', 'Null']

In [33]:
df_managers = filter(lambda x: x != 'Null', df_managers)
df_managers = list(map(int, df_managers))
df_managers

[3, 4]

Собереме словарь, где ключ id менеджера, а значение - его зарплата

In [61]:
dc = {}
for i in df_managers:
  dc[i] = df[df.id == i]['Salary'].sum()

In [62]:
for k,v in dc.items():
  print(f'key= {k}, value= {v}')

key= 3, value= 60
key= 4, value= 90


Добавим ячейку с зарплатой

In [72]:
df['manager_sallary'] = df['Managerld']
df['manager_sallary'] = df['manager_sallary'].apply(lambda x: dc[int(x)] if x != 'Null' else 'Null')
df

,id,Name,Salary,Managerld,manager_sallary
0,1,Joe,70,3,60
1,2,Henry,80,4,90
2,3,Sam,60,Null,Null
3,4,Max,90,Null,Null


Выведем имя сотрудника, у которого зарплата выше зарплаты его менеджера

In [98]:
df_employees_maximum_sallary = df.where(df['Salary'] > df['manager_sallary'])['Name'].dropna()
df_employees_maximum_sallary.iloc[0]

'Joe'